<a href="https://colab.research.google.com/github/rmrosi/Construindo-Agentes---Alura/blob/main/javasensei_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎓 JavaSensei: O Corretor Inteligente que avalia o seu Código com o Estilo do seu Professor

Este notebook permite que você utilize o agente inteligente JavaSensei para avaliar automaticamente códigos Java com base em critérios personalizados definidos por um professor.

📅 Última atualização: 17/05/2025


## 📝 Instalar o Google Gemini

In [1]:

!pip -q install google-genai

## 📝 Configura a API Key do Google Gemini

In [2]:
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

## 📝 Configura o cliente da SDK do Gemini

In [3]:
from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

## 📝 Instalar Framework ADK de agentes do Google

In [4]:
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [5]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

## 📝 Função que envia uma mensagem para um agente via Runner e retorna a resposta final

In [6]:
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

## 📝 Função auxiliar para exibir texto formatado em Markdown no Colab

In [7]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

## 👨‍🏫 Defina os critérios de correção do código fonte (Agente Avaliador)

In [16]:
################################################################################
# --- Agente 1: Avaliador do código baseado em critérios pré estabelecidos --- #
################################################################################
def agente_avaliador(atividade, codigo_fonte):
  avaliador = Agent(
      name = "agente_avaliador",
      model = "gemini-2.0-flash",
      instruction="""
      Você é um especialista em avaliar código fonte escrito na linguagem de programação Java. Será fornecido um texto com a descrição da atividade que deverá ser resolvida através do programa em Java.
      As notas devem ser mensuradas entre valores de zeroa a dez (0-10). Use usar a ferramenta de busca do Google (google_search) para recuperar as boas práticas e padrões de programação mais relevantes
      para a linguagem Java. A sua tarefa é avaliar o código fonte submetido por um aluno seguindo os critérios abaixo:
      1) verificar se o código atende às boas praticas de programação da linguagem Java (padrões de programação);
      2) O código fonte deverá conter no mínimo a definição de uma classe e o método principal (main). Se na descrição da atividade for solicitado métodos (funções ou procedimentos) e classes
         complementares, a não existência de um ou mais destes elementos deverá ser penalizada.
      3) cada um dos itens abaixo (estilo, funcionalidades atendidas, clareza e completude) devem ter notas variando de 0 a 10, mas ponderadas com os seguintes pesos para definir a nota final:
         a) estilo: 0.1;
         b) funcionalidades atendidas: 0.7;
         c) clareza: 0.1;
         d) completude: 0.1
      4) você deve adotar as seguinte penalidades que somadas não devem ultrapassar a nota máxima 10. Caso ultrapasse, normalize para o valor 10:
         a) falta de comentarios: 0.5;
         b) variaveis confusas, ou seja, nomenclatura que não está relacionado ao dado a ser armazenado, sem obedecer ao padrão camelCase, dispostas de forma desorganizada,
            variáveis desnecessárias ou redundantes: 1.0;
         c) estrutura do código ruim, sem obedecer a identação e aos padrões legibilidade e fuidez do código: 1.5;
         d) código incompleto: 2.0;
         e) codigo não compila: 2.0;
         f) variáveis não definidas, mas necessários no código: 1.0;
         g) variáveis utilizadas, mas não declarada ou mal declaradas (exemplo: deveria ser do tipo float, mas declarou como int): 0.5;
         h) estruturas inadequadas, por exemplo: usar uma estrutura "for" para uma condição de parada que não se conhece previamente ou uma estrutura de repetição "while"
            para uma repetição que já se conhece previamente a quantidade de iterações; ou ainda o uso de condicionais "if" sem o uso de "else" forçando as testas todas
            condições mesmo que a condição procurada já tenha sido testada: 1.0;
         i) erros de sintaxe (operadores de atribuição, relacional, lógico e aritmético, estruturas condicionais ou de repetição): 1.5;
         j) Mensagens de entrada e saida de dados enviadas para o usuário do programa não amigáveis ou claras: 0.5

       Se o código não compilar, mesmo assim, você deve realizar uma avaliacao parcial baseada nos seguintes critérios:
       1) quantidades de variaveis corretas e bem definidas: 0.2;
       2) uso correto das estrutura condicionais, de repetição estrutura  ou de armazenamento como, por exemplo, vetores e matrizes: 0.4;
       3) uso adequado de operadores de atribuição, aritméticos, lógicos e relacionais: 0.3;
       4) logica parcialmente correta de seguir a tabela abaixo:
          a) "0% a 10%": 0.0;
          b) "11% a 40%": 2.0;
          c) "41% a 60%": 4.0;
          d) "61% a 80%": 6.0;
          e) "81% a 99%": 8.0

         As mensagens para o aluno devem ser educativos e gentis. O feedback dado após a avaliação pode serguir os exemplos abaixo:
         1) feedback_positivo: "Você está no caminho certo! Algumas partes da sua lógica fazem sentido.",
         2)feedback_negativo: "Seu código possui muitas falhas que precisam ser revistas. Verifique a estrutura e a sintaxe."
      """,
      description="Agente que conhece as regras de pontuação do código Java submetido",
      tools=[google_search]
  )

  entrada_do_agente_avaliador = f"Atividade {atividade} \nCódigo fonte: {codigo_fonte}"

  resultado_avaliacao = call_agent(avaliador, entrada_do_agente_avaliador)
  return resultado_avaliacao

## 👨‍🏫 Normaliza a nota dada pelo agente avaliador baseado no estilo de correção do professor (Agente Normalizador)

In [19]:
################################################################################
# --- Agente 1: Avaliador do código baseado em critérios pré estabelecidos --- #
################################################################################
def agente_normalizador(atividade, codigo_fonte, resultado_avaliacao):
  normalizador = Agent(
      name = "agente_normalizador",
      model = "gemini-2.0-flash",
      instruction="""
      Você recebeu o resultado da avaliação de um especialista em avaliar código fonte escrito na linguagem de programação Java. O especialista fez a avaliação da atividade utilizando as
      seguintes regras: as notas foram mensuradas entre valores de zeroa a dez (0-10) utilizando os critérios abaixo:
      1) verificar se o código atende às boas praticas de programação da linguagem Java (padrões de programação);
      2) O código fonte deverá conter no mínimo a definição de uma classe e o método principal (main). Se na descrição da atividade for solicitado métodos (funções ou procedimentos) e classes
         complementares, a não existência de um ou mais destes elementos deverá ser penalizada.
      3) cada um dos itens abaixo (estilo, funcionalidades atendidas, clareza e completude) devem ter notas variando de 0 a 10, mas ponderadas com os seguintes pesos para definir a nota final:
         a) estilo: 0.1;
         b) funcionalidades atendidas: 0.7;
         c) clareza: 0.1;
         d) completude: 0.1
      4) você deve adotar as seguinte penalidades que somadas não devem ultrapassar a nota máxima 10. Caso ultrapasse, normalize para o valor 10:
         a) falta de comentarios: 0.5;
         b) variaveis confusas, ou seja, nomenclatura que não está relacionado ao dado a ser armazenado, sem obedecer ao padrão camelCase, dispostas de forma desorganizada,
            variáveis desnecessárias ou redundantes: 1.0;
         c) estrutura do código ruim, sem obedecer a identação e aos padrões legibilidade e fuidez do código: 1.5;
         d) código incompleto: 2.0;
         e) codigo não compila: 2.0;
         f) variáveis não definidas, mas necessários no código: 1.0;
         g) variáveis utilizadas, mas não declarada ou mal declaradas (exemplo: deveria ser do tipo float, mas declarou como int): 0.5;
         h) estruturas inadequadas, por exemplo: usar uma estrutura "for" para uma condição de parada que não se conhece previamente ou uma estrutura de repetição "while"
            para uma repetição que já se conhece previamente a quantidade de iterações; ou ainda o uso de condicionais "if" sem o uso de "else" forçando as testas todas
            condições mesmo que a condição procurada já tenha sido testada: 1.0;
         i) erros de sintaxe (operadores de atribuição, relacional, lógico e aritmético, estruturas condicionais ou de repetição): 1.5;
         j) Mensagens de entrada e saida de dados enviadas para o usuário do programa não amigáveis ou claras: 0.5

       Se o código não compilar, mesmo assim, você deve realizar uma avaliacao parcial baseada nos seguintes critérios:
       1) quantidades de variaveis corretas e bem definidas: 0.2;
       2) uso correto das estrutura condicionais, de repetição estrutura  ou de armazenamento como, por exemplo, vetores e matrizes: 0.4;
       3) uso adequado de operadores de atribuição, aritméticos, lógicos e relacionais: 0.3;
       4) logica parcialmente correta de seguir a tabela abaixo:
          a) "0% a 10%": 0.0;
          b) "11% a 40%": 2.0;
          c) "41% a 60%": 4.0;
          d) "61% a 80%": 6.0;
          e) "81% a 99%": 8.0

         As mensagens para o aluno devem ser educativos e gentis. O feedback dado após a avaliação pode serguir os exemplos abaixo:
         1) feedback_positivo: "Você está no caminho certo! Algumas partes da sua lógica fazem sentido.",
         2)feedback_negativo: "Seu código possui muitas falhas que precisam ser revistas. Verifique a estrutura e a sintaxe."

         A sua função é normalisar a nota dada pelo especialista (resultado da avaliação) se baseando no estilo de avaliação do professor X. Para te ajudar, estou listando abaixo algumas avaliações realizadas
         pelo professor X. A lista está posta com a descrição da atividade, o código fonte Java fornecido pelo aluno e a nota final dada pelo professor X:
         Avaliação 1:
          a) descrição da atividade:"Uma parede em formato retangular, cuja altura é hp (altura da parede) e a largura lp (largura da parede) precisa ser coberta por azulejos também retangulares.
          O azulejo retangular tem dimensões ha (altura do azulejo) e la (largura do azulejo). Escreva um programa que leia as quatro medidas hp, lp, ha e la, calcule e imprima quanto azulejos
          com as medidas dadas são necessários para cobrir a parede dada."
          b) código fonte Java fornecido pelo aluno:
          "import java.util.Scanner;

          public class prova {

            public static void main(String[] args) {
                Scanner scan = new Scanner(System.in);
                int calcAreaParede;
                int calcAzuleijo;
                int hp;
                int lp;
                int ha;
                int la;
                int resultado;

                //altura parede
                System.out.println("Digite a altura da parede");
                hp =scan.nextInt();

                // largura p
                System.out.println("Digite a largura da parede");
                lp = scan.nextInt();

                //altursa azu
                System.out.println("Digite a altura do azuleijo");
                ha = scan.nextInt();

                //largura azul
                System.out.println("Digite a largura do azuleijo");
                la = scan.nextInt();

                //CALCULO DA AREA
                calcAreaParede = hp*lp;
                calcAzuleijo= ha*la;

                resultado = calcAreaParede /calcAzuleijo;

                if(calcAzuleijo >calcAreaParede) {

                  System.out.println("o azuleijo e muito grande e nao cabe na parede");
                }
                else{
                System.out.println("Cabem na parede " +resultado+ " Azuleijo");}
            }
          }"
          c) nota final dada pelo professor X:8.7
          Avaliação 2:
          a) descrição da atividade:"Escreva um programa que, com base em uma temperatura em graus celsius, a converta e exiba em Kelvin (K), Réaumur (Re), Rankine (Ra) e Fahrenheit (F), seguindo as fórmulas:
              F = C * 1.8 + 32;
              K = C + 273.15;
              Re = C * 0.8;
              Ra = C * 1.8 + 32 + 459.67
          "
          b) código fonte Java fornecido pelo aluno:
          "
          public class Provajava {
            public static void main(String[] args) {

                int graus = 10;

                double F = graus * 1.8 + 32;
                double K = graus + 273.15;
                double Re = graus * 0.8;
                double Ra = graus  * 1.8 + 32 + 459.67;

                System.out.println("Temperatura em Fahrenheit: " + F);
                System.out.println("Temperatura em Kelvin: " + K);
                System.out.println("Temperatura em Réaumur: " + Re);
                System.out.println("Temperatura em Rankine: " + Ra);
            }
          }
          "
          c) nota final dada pelo professor X: 7.8
          Avaliação 3:
          a) descrição da atividade:"Escreva um programa que, com base em uma temperatura em graus celsius, a converta e exiba em Kelvin (K), Réaumur (Re), Rankine (Ra) e Fahrenheit (F), seguindo as fórmulas:
              F = C * 1.8 + 32;
              K = C + 273.15;
              Re = C * 0.8;
              Ra = C * 1.8 + 32 + 459.67
          "
          b) código fonte Java fornecido pelo aluno:
          "
          import java.util.Scanner;

          public class Questao1 {
            public static void main(String[] args) {

                Scanner scan=new Scanner(System.in);
                float c,f,k,re,ra;
                System.out.print("Insira o valor em  graus celsius:");
                c=scan.nextFloat();
                f = (float) (c * 1.8 + 32);
                k = (float) (c + 273.15);
                re = (float) (c * 0.8);
                ra = (float) (c * 1.8 + 32 + 459.67);

                System.out.println("A temperatura em Kelvin "+k);
                System.out.println("A temperatura em Réaumur "+re);
                System.out.println("A temperatura em Rankine "+ra);
                System.out.println("A temperatura em Fahrenheit "+f);

                scan.close();
            }
          }
          "
          c) nota final dada pelo professor X: 10.0
          Avaliação 4:
          a) descrição da atividade:"Crie um algoritmo que leia o valor do salário mínimo e o valor do salário de um usuário,
          calcule a quantidade de salários mínimos esse usuário ganha e imprima o resultado (Salário Mínimo=R$1.320,00)."
          b) código fonte Java fornecido pelo aluno:
          "import java.util.Scanner;

          public class Prova {
            public static void main(String[] args) {
                Scanner scanner = new Scanner(System.in);
                System.out.println("Qual é seu Salário");
                double salario = scanner.nextDouble();
                System.out.println("A Quantidade de vezes o salário mínimos ganhos é: "+ salario/1320);
            }
          }"
          c) nota final dada pelo professor X:10.0
          Avaliação 5:
          a) descrição da atividade:
          "Escrever um algoritmo que leia:
          a porcentagem do IPI a ser acrescido no valor das peças
          o código da peça 1, valor unitário da peça 1, quantidade de peças 1
          o código da peça 2, valor unitário da peça 2, quantidade de peças 2
          O algoritmo deve calcular o valor total a ser pago e apresentar o resultado das duas peças como no exemplo abaixo:
          Código           Qtde           Valor Unit         IPI         Total c/ IPI
            1               10               245,50          10%            2700,50
           33               10                50,00          10%             550,00
                                                            Total Geral:    3250,50
          Fórmula : (valor*quant)*(IPI/100 + 1)"
          b) código fonte Java fornecido pelo aluno:
          "import java.util.Scanner;

          public class Teste01 {

            public static void main(String[] args) {
                Scanner scan = new Scanner (System.in);
                int ipi;
                int cod1;
                double valorUnit1;
                int qunt1;
                int cod2;
                double valorUnit2;
                int qunt2;
                double totalIpi1=0;
                double totalIpi2=0;
                double soma;
                System.out.println("Entre com o valor do IPI: ");
                ipi = scan.nextInt();

                System.out.println("Entre com o Código da peça: ");
                cod1 = scan.nextInt();

                System.out.println("Entre com o valor unitário: ");
                valorUnit1 = scan.nextDouble();

                System.out.println("Entre com a quantidade de peças: ");
                qunt1 = scan.nextInt();

                System.out.println("Entre com o valor do IPI: ");
                ipi = scan.nextInt();

                System.out.println("Entre com o Código da peça: ");
                cod2 = scan.nextInt();

                System.out.println("Entre com o valor unitário: ");
                valorUnit2 = scan.nextDouble();

                System.out.println("Entre com a quantidade de peças: ");
                qunt2 = scan.nextInt();

                totalIpi1 = (valorUnit1*qunt1)*(ipi/100 + 1);
                totalIpi2 = (valorUnit2*qunt2)*(ipi/100 + 1);
                soma = totalIpi1 + totalIpi2;
                System.out.println(soma);

            }

          }"
          c) nota final dada pelo professor X:4.7
      """,
      description="Agente que conhece o estilo do professor X para a correção do código Java"
  )

  entrada_do_agente_normalizador = f"Atividade {atividade} \nCódigo fonte: {codigo_fonte} \nResultado da Avaliação: {resultado_avaliacao}"

  avaliacao_normalizada = call_agent(normalizador, entrada_do_agente_normalizador)
  return avaliacao_normalizada

## 👨‍🏫 Orienta o aluno em como melhorar o seu aprendizado (Agente Orientador)

In [22]:
###########################################
# --- Agente 3: Orientador de Estudos --- #
###########################################
def agente_orientador(atividade, codigo_fonte, resultado_avaliacao):
    orientador = Agent(
        name="agente_orientador",
        model='gemini-2.0-flash',
        instruction="""
            Você é um professor Orientador e Educador meticuloso, especializado em ensino de programação, com foco na linguagem Java.
            Por ter um público jovem, universitários entre 18 e 25 anos, use um tom de escrita adequado.
            Avalie o resultado da avaliação abaixo sobre o código Java da atividade avaliada, verificando os pontos de melhorias, os principais erros, as questões técnicas e padrões da linguagem Java e
            considerando todo o seu conhecimento, as boas práticas e metodologias de aprendizagem pesquisadas a partir da ferramenta de busca do Google (google_search), faça um conjunto de considerações e
            sugestões de estudo para que o aluno possa melhorar o seu código e o seu aprendizado da linguagem Java.
            Faça também sugestões de tutoriais, artigos, livros, vídeo-aulas e cursos online/web que possam potencializar o aprendizado do aluno. Considere o grau de dificuldade de aprendizado do aluno,
            baseado no seu resultado/nota para indicar um conjunto de material de estudo mais adequado ao nível de dificuldade do aluno.
            Se o aluno obteve nota máxima, faça um elogio e sugestões de material para que ele possa avançar em profundidade noconhecimento da linguagem de programação Java.
            """,
        description="Agente orientador de aprendizado da linguagem Java.",
        tools=[google_search]
    )
    entrada_do_agente_revisor = f"Atividade {atividade} \nCódigo fonte: {codigo_fonte} \nResultado da Avaliação: {resultado_avaliacao}"
    # Executa o agente
    texto_revisado = call_agent(orientador, entrada_do_agente_revisor)
    return texto_revisado

## 🤖 Avaliação do código pelo agente JavaSensei

In [23]:
print("🚀 Iniciando o Sistema de Avaliação de Código Java com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
atividade_1 = "Escreva um código java que imprima na tela a mensagem 'Olá, mundo!'"

codigo_java_1 = """public class HelloWorld {
    public static void main(String[] args) {
        System.out.println("Olá, mundo!");
    }
}
"""
atividade = "Leia o valor de um saldo e informe o saldo e o seu valor com reajuste de 1%."
codigo_java = """
import java.util.Scanner;
public class Saldo {

   public static void main(String[] args) {

   Scanner in = new Scanner(System.in);

      System.out.println("digite o valor depositado");

   float saldo = in.nextFloat();

      System.out.println("imprimindo saldo: "+saldo);
      in.close();
   }

}
"""
#atividade = input("❓ Por favor, digite o ATIVIDADE relacionada ao código Java que será avaliado: ")


# Inserir lógica do sistema de agentes
if not atividade:
  print("Você esqueceu de digitar a atividade!")
else:
  #codigo_java = input("❓ Por favor, digite o CÓDIGO JAVA que será avaliado: ")
  if not codigo_java:
    print("Você esqueceu de digitar o código fonte!")
  else:
    print("Maravilha! Vamos então avaliar o seu código fonte!")

    resultado_avaliacao = agente_avaliador(atividade, codigo_java)
    print("\n--- 📝 Resultado do Agente 1 (Avaliador) ---\n")
    display(to_markdown(resultado_avaliacao))
    print("--------------------------------------------------------------")

    avaliacao_normalizada = agente_normalizador(atividade, codigo_java, resultado_avaliacao)
    print("\n--- 📝 Resultado do Agente 2 (Normalizador) ---\n")
    display(to_markdown(avaliacao_normalizada))
    print("--------------------------------------------------------------")

    orientacao_recebida = agente_orientador(atividade, codigo_java, resultado_avaliacao)
    print("\n--- 📝 Resultado do Agente 3 (Orientador) ---\n")
    display(to_markdown(orientacao_recebida))
    print("--------------------------------------------------------------")

🚀 Iniciando o Sistema de Avaliação de Código Java com 4 Agentes 🚀
Maravilha! Vamos então avaliar o seu código fonte!

--- 📝 Resultado do Agente 1 (Avaliador) ---



> Ok, vou avaliar o código Java que você forneceu para calcular o reajuste de 1% sobre um saldo.
> 
> 
> ## Avaliação do Código
> 
> Com base nos critérios fornecidos, aqui está a avaliação do seu código:
> 
> **1. Estilo (Peso: 0.1):**
> 
> *   **Nota:** 7.0
> *   **Comentários:** O código é relativamente simples, mas carece de comentários explicando o propósito do programa e das variáveis. A nomenclatura `in` para o objeto `Scanner` é comum, mas poderia ser mais descritiva em projetos maiores. A identação está correta, o que ajuda na legibilidade.
> 
> **2. Funcionalidades Atendidas (Peso: 0.7):**
> 
> *   **Nota:** 4.0
> *   **Comentários:** O código recebe um valor do usuário e imprime o saldo, mas não realiza o cálculo do reajuste de 1%, conforme solicitado na atividade.
> 
> **3. Clareza (Peso: 0.1):**
> 
> *   **Nota:** 8.0
> *   **Comentários:** O código é fácil de entender devido à sua simplicidade e boa identação. As mensagens para o usuário são claras, embora possam ser mais amigáveis.
> 
> **4. Completude (Peso: 0.1):**
> 
> *   **Nota:** 4.0
> *   **Comentários:** O código está incompleto, pois não implementa a lógica para calcular e exibir o saldo com o reajuste de 1%.
> 
> ### Penalidades:
> 
> *   Falta de comentários: -0.5
> *   Código incompleto: -2.0
> 
> ### Cálculo da Nota Final:
> 
> Nota final = (0.1 \* 7.0) + (0.7 \* 4.0) + (0.1 \* 8.0) + (0.1 \* 4.0) - 0.5 - 2.0 = 0.7 + 2.8 + 0.8 + 0.4 - 0.5 - 2.0 = 1.2
> 
> **Nota Final: 1.2**
> 
> ### Sugestões:
> 
> 1.  **Implementar o cálculo do reajuste:** Adicione a lógica para calcular o reajuste de 1% sobre o saldo.
> 2.  **Exibir o saldo reajustado:** Imprima o valor do saldo após o reajuste.
> 3.  **Adicionar comentários:** Inclua comentários para explicar o propósito do código e das variáveis, melhorando a compreensão.
> 4.  **Melhorar as mensagens ao usuário:** Torne as mensagens de entrada e saída mais amigáveis e informativas.
> 
> ### Código revisado
> 
> 
> ```java
> import java.util.Scanner;
> 
> public class Saldo {
> 
>     public static void main(String[] args) {
> 
>         // Cria um objeto Scanner para receber dados do usuário
>         Scanner input = new Scanner(System.in);
> 
>         // Solicita ao usuário que digite o valor do saldo
>         System.out.println("Digite o valor do saldo:");
> 
>         // Lê o valor do saldo digitado pelo usuário
>         float saldo = input.nextFloat();
> 
>         // Calcula o reajuste de 1% sobre o saldo
>         float reajuste = saldo * 0.01f;
>         float novoSaldo = saldo + reajuste;
> 
>         // Imprime o saldo original
>         System.out.println("Saldo original: " + saldo);
> 
>         // Imprime o saldo com o reajuste de 1%
>         System.out.println("Saldo com reajuste de 1%: " + novoSaldo);
> 
>         // Fecha o objeto Scanner
>         input.close();
>     }
> }
> ```
> 
> 
> **Feedback:** Seu código possui algumas falhas que precisam ser revisadas. Verifique a lógica e a implementação do cálculo do reajuste. Adicione os comentários explicativos.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 2 (Normalizador) ---



> A nota fornecida pelo especialista (1.2) parece muito baixa em relação ao que o professor X geralmente atribui. Analisando os exemplos fornecidos, o professor X tende a ser mais generoso, mesmo com códigos que não estão totalmente completos ou que possuem pequenas falhas.
> 
> No caso presente, o código cumpre parcialmente o que foi solicitado: ele recebe a entrada do usuário e imprime o saldo. A principal falha é não calcular o reajuste. No entanto, a estrutura básica do programa está correta.
> 
> Aplicando o estilo de avaliação do professor X, podemos considerar o seguinte:
> 
> *   **O código compila e executa:** Isso já é um ponto positivo.
> *   **Entrada e Saída:** A entrada e saída de dados funcionam, embora a saída seja incompleta.
> *   **Lógica:** A lógica está *parcialmente* correta (receber o saldo).
> *   **Estrutura:** A estrutura do código é simples e compreensível.
> 
> Considerando que o código não implementa o reajuste, mas possui uma estrutura funcional, e seguindo a lógica das avaliações anteriores do professor X, uma nota mais adequada seria em torno de 6.5. Abaixo está o raciocínio para chegar a essa nota, levando em consideração as penalidades e a avaliação parcial:
> 
> 1.  **Nota Base:** Começamos com uma nota base de 7.0 (considerando que o código compila, tem entrada/saída funcionando e uma estrutura básica).
> 
> 2.  **Penalidades:**
>     *   Falta do cálculo do reajuste (funcionalidade principal não atendida): -1.0
>     *   Falta de comentários explicativos: -0.5
> 
> 3.  **Ajuste:** 7.0 - 1.0 - 0.5 = 5.5
> 
> 4.  **Consideração da Lógica Parcialmente Correta:** A lógica para receber o valor está correta. A falta do cálculo impacta a nota, mas não a zera, visto que parte da atividade foi entregue.
> 
> 5.  **Comparativo com os exemplos:** O código é melhor que o do exemplo 5 (nota 4.7) onde há vários problemas. Se assemelha mais ao código do exemplo 2, onde a atividade foi cumprida, mas de forma muito simplificada, recebendo nota 7.8.
> 
> **Nota Final Normalizada: 6.5**
> 
> **Feedback para o Aluno:**
> 
> "Você está no caminho certo! A estrutura do seu código está boa e ele recebe corretamente o valor do saldo. Para melhorar, adicione o cálculo do reajuste de 1% e exiba o novo saldo. Incluir alguns comentários para explicar cada parte do código também ajudará na clareza. Continue praticando!"
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 3 (Orientador) ---



> Olá! Tudo bem? Analisando a sua atividade, vejo que você tem um bom começo, mas ainda há espaço para melhorias significativas. A nota 1.2 reflete que alguns pontos essenciais da atividade não foram implementados corretamente. Mas não se preocupe, estou aqui para te ajudar a entender onde você pode melhorar e como você pode aprofundar seus conhecimentos em Java.
> 
> ### Considerações e Sugestões de Estudo
> 
> 1.  **Implementação do Cálculo do Reajuste:**
>     *   O ponto crucial que faltou foi o cálculo do reajuste de 1%. Para calcular um reajuste percentual, você precisa multiplicar o saldo pelo percentual desejado e somar ao saldo original. No seu caso, 1% é representado por 0.01.
> 
> 2.  **Exibição do Saldo Reajustado:**
>     *   Além de calcular o reajuste, é importante exibir o resultado para o usuário. Isso mostra que o programa realmente fez o que foi proposto.
> 
> 3.  **Comentários no Código:**
>     *   Comentários são importantes para explicar o que cada parte do código faz. Eles ajudam outras pessoas (e você mesmo no futuro) a entenderem a lógica do programa. No seu código revisado, já adicionei alguns comentários para te dar um exemplo.
> 
> 4.  **Melhoria das Mensagens ao Usuário:**
>     *   As mensagens para o usuário podem ser mais amigáveis e informativas. Em vez de apenas "digite o valor depositado", você pode usar algo como "Por favor, insira o valor do saldo inicial:".
> 
> ### Sugestões de Materiais de Estudo
> 
> Considerando que você precisa fortalecer a lógica de programação e a implementação em Java, sugiro os seguintes materiais:
> 
> 1.  **Tutoriais e Cursos Online:**
>     *   **Curso em Vídeo de Java do Gustavo Guanabara:** Excelente para iniciantes, aborda desde o básico até conceitos mais avançados. A didática do professor é muito boa e o curso é gratuito no YouTube.
>         *   Link: [https://www.cursoemvideo.com/curso/java-basico/](https://www.cursoemvideo.com/curso/java-basico/)
>     *   **Alura:** Oferece cursos de Java com foco em desenvolvimento web e empresarial. A plataforma é paga, mas vale o investimento pela qualidade do conteúdo e suporte.
>         *   Link: [https://www.alura.com.br/cursos-online-java](https://www.alura.com.br/cursos-online-java)
>     *   **Udemy:** Possui diversos cursos de Java, desde o básico até tópicos avançados como Spring Framework. Verifique as avaliações e escolha um curso que se adapte ao seu estilo de aprendizado.
>         *   Link: [https://www.udemy.com/topic/java/](https://www.udemy.com/topic/java/)
> 
> 2.  **Livros:**
>     *   **"Java: Como Programar" de Paul Deitel e Harvey Deitel:** Um livro completo que aborda todos os aspectos da linguagem Java, desde os fundamentos até tópicos avançados.
>     *   **"Effective Java" de Joshua Bloch:** Focado em boas práticas de programação em Java, ideal para quem já tem alguma experiência e quer escrever código de alta qualidade.
> 
> 3.  **Artigos e Documentação:**
>     *   **Documentação Oficial da Oracle Java:** A fonte mais confiável para informações detalhadas sobre a linguagem Java.
>         *   Link: [https://docs.oracle.com/en/java/](https://docs.oracle.com/en/java/)
>     *   **Artigos da Digital Ocean sobre Java:** Oferecem guias práticos e tutoriais sobre diversos tópicos de Java.
>         *   Link: [https://www.digitalocean.com/community/tutorials?q=java](https://www.digitalocean.com/community/tutorials?q=java)
> 
> ### Próximos Passos
> 
> 1.  **Revise o Código:**
>     *   Use o código revisado que forneci como base e tente entender cada linha. Veja como o cálculo do reajuste foi implementado e como o resultado é exibido.
> 
> 2.  **Pratique:**
>     *   A melhor forma de aprender é praticando. Tente modificar o código para calcular reajustes diferentes (por exemplo, 5%, 10%).
>     *   Crie pequenos programas para resolver problemas simples. Isso vai te ajudar a fixar o conhecimento.
> 
> 3.  **Estude a Lógica de Programação:**
>     *   Se você está com dificuldades na lógica, procure exercícios e materiais sobre algoritmos e estruturas de dados. Isso vai te dar uma base sólida para programar em qualquer linguagem.
> 
> 4.  **Use Recursos Online:**
>     *   Aproveite os cursos e tutoriais que sugeri. Eles vão te dar uma visão mais completa da linguagem Java e te ajudar a resolver problemas mais complexos.
> 
> Lembre-se, aprender a programar leva tempo e dedicação. Não desanime com os erros e continue praticando. Se precisar de ajuda, pode me perguntar! 😊
> 


--------------------------------------------------------------
